In [2]:
%pip install --upgrade --quiet langchain-community langchain langchain-openai faiss-cpu beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# https://python.langchain.com/docs/versions/migrating_chains/retrieval_qa/

In [ ]:
# %pip install -qU langchain-mistralai
from langchain_mistralai import MistralAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
import time
from helper_funcs import read_txt_file_to_string, get_api_key_url

API_KEY, API_URL = get_api_key_url(client="MISTRAL")

embeddings = MistralAIEmbeddings(
    model="mistral-embed",
    api_key=API_KEY    
)

splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)

doc1 = read_txt_file_to_string("documents/doc1.txt")[:10]
doc2 = read_txt_file_to_string("documents/doc2.txt")[:10]

documents = [
    Document(page_content=doc1, metadata={"source": "doc1.txt"}),
    Document(page_content=doc2, metadata={"source": "doc2.txt"}),
]
splits = splitter.split_documents(documents)

# --- Initialize or load Chroma DB --- https://python.langchain.com/docs/integrations/vectorstores/chroma/
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",
)

def safe_add_documents(vector_store, docs, delay=10):
    print(docs)
    vector_store.add_documents(docs)
    time.sleep(delay)


safe_add_documents(vector_store, splits)
retriever = vector_store.as_retriever()

# Retrieve the most similar text
retrieved_documents = retriever.invoke("What is LangChain?")

# show the retrieved document's content
retrieved_documents[0].page_content

print("✅ Vectorstore created with Mistral embeddings!")


An error occurred with MistralAI
Traceback (most recent call last):
  File "c:\Users\KoenFolkers\dev\LLM_text_comparison\venv\Lib\site-packages\httpx\_transports\default.py", line 101, in map_httpcore_exceptions
    yield
  File "c:\Users\KoenFolkers\dev\LLM_text_comparison\venv\Lib\site-packages\httpx\_transports\default.py", line 250, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\KoenFolkers\dev\LLM_text_comparison\venv\Lib\site-packages\httpcore\_sync\connection_pool.py", line 256, in handle_request
    raise exc from None
  File "c:\Users\KoenFolkers\dev\LLM_text_comparison\venv\Lib\site-packages\httpcore\_sync\connection_pool.py", line 236, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\KoenFolkers\dev\LLM_text_comparison\venv\Lib\site-packages\httpcore\_sync\connection.py", line 103, in handle_request
    return self._connection.handle_r

[Document(metadata={'source': 'doc1.txt'}, page_content='Document A'), Document(metadata={'source': 'doc2.txt'}, page_content='Document B')]


LocalProtocolError: Illegal header value b'Bearer '

In [ ]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.embeddings import HuggingFaceEmbeddings

llm = HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.2")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
# https://python.langchain.com/docs/tutorials/rag/

In [ ]:
# % pip install langchain-google-genai
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
import time
from helper_funcs import read_txt_file_to_string, get_api_key_url
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from pydantic.types import SecretStr

API_KEY, API_URL = get_api_key_url(client="GEMINI", return_as_secret_strs=True)

# --- Gemini model & embeddings ---
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2, google_api_key=API_KEY)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=API_KEY)
splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)

doc1 = read_txt_file_to_string("documents/doc1.txt")
doc2 = read_txt_file_to_string("documents/doc2.txt")

documents = [
    Document(page_content=doc1, metadata={"source": "doc1.txt"}),
    Document(page_content=doc2, metadata={"source": "doc2.txt"}),
]
splits = splitter.split_documents(documents)

# --- Initialize or load Chroma DB --- https://python.langchain.com/docs/integrations/vectorstores/chroma/
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",
)

def safe_add_documents(vector_store, docs, delay=5):
    for d in docs:
        try:
            vector_store.add_documents([d])
            print("Added:", d.metadata)
        except Exception as e:
            print("Error:", e)
        time.sleep(delay)

safe_add_documents(vector_store, splits)
retriever = vector_store.as_retriever()

query = "What is an autonomous agent?"
retrieved = retriever.invoke(query)


Error: Error embedding content: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0 [violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerDayPerUserPerProjectPerModel-FreeTier"
}
violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerMinutePerUserPerProjectPerModel-FreeTier"
}
violations {


GoogleGenerativeAIError: Error embedding content: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0 [violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerMinutePerProjectPerModel-FreeTier"
}
violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerMinutePerUserPerProjectPerModel-FreeTier"
}